# <span style='color:black'> Import lib </span>

In [1]:
from shutil import copyfile
from tkinter import *
from PIL import *

import os.path
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch.optim as optim
import numpy as np
import PIL.Image
#import random
import time
#from random import randint
from Model.Funtion_Bank import *

global btn_txt_matrix
global btn_matrix
global btn_refresh_matrix
global Button
global Current_play_ground

# <span style='color:black'> Define Agent </span>

In [2]:
import numpy as np

class agent_a:
    def __init__(self, perception, state):
        self.perception = perception
        self.state = state

        #print(self.verify_Matrix.argmax())
    def restart(self):
        print(self.input_string)
        return('abc')
    def next_action(self):
        print('next action')


# <span style='color:black'> define file_path and parameters </span>

# <span style='color:black'> Game engine </span>

In [3]:
def play_ground(row, column, wall_generated, target_generated, agent_a, agent_b):
    rows = row * 3
    columns = column * 3
    init_ground = torch.tensor([[0 for x in range(columns)] for y in range(rows)], dtype=torch.float)
    wall_generated.append([row, column, row, column * 2])
    wall_generated.append([row, column, row * 2, column])
    wall_generated.append([row * 2, column, row * 2, column * 2])
    wall_generated.append([row, column * 2, row * 2, column * 2])
    for line in wall_generated:
        for i in range(line[0], line[2] + 1):
            for j in range(line[1], line[3] + 1):
                init_ground[i][j] = 1
    for point in target_generated:
        init_ground[point[0]][point[1]] = 2
    for point in agent_a:
        init_ground[point[0]][point[1]] = 3
    for point in agent_b:
        init_ground[point[0]][point[1]] = 4
    return init_ground

In [ ]:
def DL_training_model():
    indicator = 0
    learning_rate = 0.1
    epoch_size = 100
    steps_for_printing_out_loss = 10
    #cropped_perspective = np.array([4,5,6])
    input_data = data_input(input_file_path)
    score_data = data_input(score_file_path)
    input = input_data
    #print(input)
    target = score_data
    net = Net()
    loss_functioin = nn.MSELoss()
    optimizer = optim.SGD(net.parameters(), lr = learning_rate)
    if indicator == 1:
        state_dict_last_version = torch.load('Model/model_latest_version.pt')['state_dict']
        net.load_state_dict(state_dict_last_version)
    for i in range(1,epoch_size + 1):
        optimizer.zero_grad()
        output = net(input)
        output[target == -2] = -2
        #output[target == -1.1] = -1.1
        loss = loss_functioin(output, target)
        loss.backward()
        if i % (steps_for_printing_out_loss) == 0:
            print('Loss (epoch: ' + str(i) + '): ' + str(loss.cpu().detach().numpy()))
        # Does the update
        optimizer.step()
    
    if indicator == 1:
        copyfile(file_name_model_latest_version, file_name_model_last_version)
    else:
        torch.save({'state_dict': net.state_dict(),'optimizer': optimizer.state_dict()}, file_name_model_last_version)
    torch.save({'state_dict': net.state_dict(),'optimizer': optimizer.state_dict()}, file_name_model_latest_version)


def DL_model(cropped_perspective, model_version, state):
    model_latest_version = Net()
    state_dict_latest_version = torch.load(file_name_model_latest_version)['state_dict']
    model_latest_version.load_state_dict(state_dict_latest_version)
    next_vision = model_latest_version(torch.tensor(cropped_perspective.reshape(252), dtype=torch.float))
    #print(next_vision)
    if ((state[0] == -1) and (state[1] == 1)) or ((state[0] == 0) and (state[1] == playground_w)):
        next_vision[0] = -3
    if ((state[0] == -1) and (state[3] == playground_w)) or ((state[0] == 0) and (state[3] == 1)):
        next_vision[2] = -3
    #print('abc')
    #print(next_vision)
    next_step = next_vision.argmax()
    action = next_step
    return action

# <span style='color:black'> Main entrance </span>

In [4]:
if __name__ == "__main__":
    row = 10
    column = 15
    no_of_wall = 10
    no_of_target = 7
    no_of_agent = 5
    #initial position
    wall_generated = wall_generation(row, column, no_of_wall)
    target_generated = target_generation(row, column, no_of_target)
    agent_a = agent_generation(row, column, no_of_agent)
    agent_b = agent_generation(row, column, no_of_agent)
    
    tensor_play_ground = play_ground(row, column, wall_generated, target_generated, agent_a, agent_b)
    tensor_play_ground_numpy = tensor_play_ground.clone()
    play_ground_numpy = np.flip(tensor_play_ground_numpy.cpu().numpy(), 0)
    
    centre_point = [15, 23]
    matrix_trancated = trancate_matrix(play_ground_numpy, centre_point, row, column)
    print(matrix_trancated)
    #21, 31
    print()
    pd.DataFrame(play_ground_numpy).to_csv('UI_sample.csv')

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 2. 0. 0. 4. 4. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 3. 1. 0. 1. 0. 2. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 3. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 2. 0. 0. 0. 2. 1. 1. 1. 0. 0. 0. 3. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 

In [5]:
#leader
#predict other agent next action
#against other agent's prediction
#communicatoin matrix
#memory
#remove some wall
#set up starting point for agent a & b
#speed & count parameter
#use an array to present each point - []
[0,1,0,0,0,0,0,0,0,0,0]
